In [1]:
from binance.websocket.um_futures.websocket_client import UMFuturesWebsocketClient

import json
import time
import datetime
import numpy as np
import pandas as pd
from collections import deque
import asyncio
import requests

pd.options.mode.copy_on_write = True

In [ ]:
# Test Net API Key and Secret
API_KEY = "b88d219ce4b7f4408ab0f51fc8be1730b264904e48e9a6ac7c7fa716e1e18dc7"
API_SECRET = "15acdeca3bb1dd761e472034c8b09bb597e165d18c69e485738ccf9a3aa4326a"

In [7]:
base_url_REST_testnet = "https://testnet.binancefuture.com"
base_url_WS_testnet = "wss://stream.binancefuture.com"

## WebSocket

In [2]:
def on_message(_, message):
    trade_data = json.loads(message)
    print(f"""
    Timestamp: {trade_data['T']}
    Price: {trade_data['p']}
    Quantity: {trade_data['q']}
    ---
    """)

In [3]:
def on_error(_, error):
    print(json.loads(error))

In [4]:
def on_close(_):
    print("WebSocket Connection Closed")

In [5]:
def on_open(_):
    print("WebSocket Connection Opened")

In [8]:
ws_client = UMFuturesWebsocketClient(stream_url=base_url_WS_testnet,
                                     on_message=on_message,
                                     on_error=on_error,
                                     on_close=on_close,
                                     on_open=on_open)

WebSocket Connection Opened


In [9]:
# Subscribe to a single symbol stream
ws_client.agg_trade(symbol="ETHUSDT", speed=3)
time.sleep(5)

Error from callback <function on_message at 0x0000018E5407E200>: 'T'
Exception in thread Thread-3:
Traceback (most recent call last):
  File "c:\Users\Alvin Lui\anaconda3\envs\humpback\Lib\site-packages\binance\websocket\binance_socket_manager.py", line 108, in _callback
    callback(self, *args)
  File "C:\Users\Alvin Lui\AppData\Local\Temp\ipykernel_15800\2435769279.py", line 4, in on_message
KeyError: 'T'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Alvin Lui\anaconda3\envs\humpback\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\Alvin Lui\anaconda3\envs\humpback\Lib\site-packages\binance\websocket\binance_socket_manager.py", line 55, in run
    self.read_data()
  File "c:\Users\Alvin Lui\anaconda3\envs\humpback\Lib\site-packages\binance\websocket\binance_socket_manager.py", line 96, in read_data
    self._callback(self.on_message, data)
  File "c:\Users\Alvin Lui\anaconda3

In [ ]:
def cal_rolling_zScore(df,
                       window_size: int=14):
    # Calculate rolling mean and rolling standard deviation
    df['Rolling Mean'] = df['Close'].rolling(window=window_size).mean()
    df['Rolling Std']  = df['Close'].rolling(window=window_size).std()
    
    # Calculate the rolling z-score
    df['Rolling Z Score'] = (df['Close'] - df['Rolling Mean']) / df['Rolling Std']
    return df.dropna()

In [ ]:
symbols = ['BTCUSDT', 'ETHUSDT']

## Choose 5m as trading interval

In [ ]:
# for symbol in symbols:
#     data[symbol] = cal_rolling_zScore(data[symbol])

In [ ]:
# spread_df = pd.DataFrame(index=data['BTCUSDT'].index.intersection(data['ETHUSDT'].index))

# # k is the ratio between BTC and ETH
# # To calculate the adjusted spread, the formula is
# # Z_X - k * Z_Y for assets X and Y

# spread_df['k'] = (data['ETHUSDT']['Rolling Std'] / data['BTCUSDT']['Rolling Std']).clip(4, 0)
# spread_df['Adjusted Spread'] = data['BTCUSDT']['Rolling Z Score'] - spread_df['k'] * data['BTCUSDT']['Rolling Z Score']

# # When the Spread is Positive, it means that BTC is overpriced.
# # Therefore, the strategy is short BTC and long ETH

## Trading Futures with API

In [ ]:
leverage = 2
buffer = deque(maxlen=100)

In [ ]:
def on_message(buffer, data):
    buffer.append(data)
    df = pd.DataFrame(buffer)
    return (buffer, df)

In [ ]:
# Change leverage
params = {
    'symbol': symbol,
    'leverage': leverage
}

response = um_futures_client.change_leverage(**params)
print(response)

In [ ]:
# # Post a new order
# params = {
#     'symbol': symbol,
#     'side': 'SELL',
#     'type': 'LIMIT',
#     'timeInForce': 'GTC',
#     'quantity': 25,
#     'price': 2540
# }

# response = um_futures_client.new_order(**params)

In [ ]:
# params = {
#     'symbol': symbol,
#     'side': 'BUY',
#     'type': 'MARKET',
#     'quantity': 1
# }

# response = um_futures_client.new_order(**params)
# print(printable_json(response))

In [ ]:
# params = {
#     'symbol': symbol,
#     'side': 'SELL',
#     'type': 'STOP_MARKET',
#     'quantity': 1,
#     'stopPrice': 2600
# }

# response = um_futures_client.new_order(**params)
# print(printable_json(response))

In [ ]:
# # Post a new order
# params = {
#     'symbol': symbol
# }

# response = um_futures_client.get_position_margin_history(**params)
# print(printable_json(response))

In [ ]:
# response = um_futures_client.get_position_risk(**params)
# print(printable_json(response))

In [ ]:
# def printable_json(response):
#     return json.dumps(response, indent = 4)